# 数据库操作shell命令

### 查看版本号

```
mongo --version
MongoDB shell version v3.6.3
git version: 9586e557d54ef70f9ca4b43c26892cd55257e1a5
OpenSSL version: OpenSSL 1.0.1u-fips  22 Sep 2016
allocator: tcmalloc
modules: none
build environment:
    distmod: 2008plus-ssl
    distarch: x86_64
    target_arch: x86_64
```  

注意: 
1. 这个命令在控制台输入,而不是mongodb交互环境中的命令
2. 一般用于安装好,检查mongodb是否安装成功

### 进入和退出mongo的shell环境

进入mongo shell环境:
```
> mongo
MongoDB shell version v3.6.3
connecting to: mongodb://127.0.0.1:27017
MongoDB server version: 3.6.3
Server has startup warnings:
...
```

退出mongo shell环境:
```
> exit
bye
```

### 在本地访问服务器上的mongo

1. 本地安装mongodb
2. 在终端输入: 
    * 方式一: `mongo ip:port/dbname -u username -p`, 回车然后输入密码,或者`mongo ip:port/dbname -u username -p passwd`,回车
    * 方式二: `mongo -u username -p passwd 192.168.0.197:27017/dbname`, 回车

其中, ip是服务器的ip, port是服务器上mongo指定的端口, dbname是你要访问的数据库, dbname不能少

### 创建管理用户

mongodb没有超级无敌用户root,只有管理用户的用户: userAdminAnyDatabase

### 查看帮助信息

```
// 查看所有数据级别的操作
> db.help()
// 查看集合级别的操作
> db.mycoll.help()
// 列举数据库命令
> db.listCommands()
```

### 显示当前正在使用的数据库

```
> db
test
```

### 显示当前数据库的用户信息

```
> show users
{
        "_id" : "admin.root",
        "user" : "root",
        "db" : "admin",
        "roles" : [
                {
                        "role" : "userAdminAnyDatabase",
                        "db" : "admin"
                }
        ]
}
```

### 显示所有数据库

```
> show dbs
hawk  0.136GB
admin    0.000GB
config   0.000GB
local    0.000GB
testdb   0.001GB
```

### 切换到指定数据库

```
> use testdb
switched to db testdb
```

### 显示当前数据库中所有的集合

```
> show collections
info
users
score
```
或者
`db.getCollectionNames()`

### 查询指定数据库统计信息

```
use dbname
db.stats()
```

### 创建一个集合

```
> db.createCollection("users")
{ "ok" : 1 }
```

或者

`db.createCollection(name, { size : ..., capped : ..., max : ... } )`

### 删除集合

`db.mycoll.drop()`

### 删除数据库

`db.dropDatabase()`

### 查询指定数据库的集合当前可用的存储空间

`db.baseSe.storageSize()`

### 查询指定数据库的集合分配的存储空间

`db.baseSe.totalSize()`